In [2]:
import pandas as pd
import re
from datetime import datetime

In [3]:
dat = pd.read_excel("../data/external/0053 Rxs no PHI March - May 2021.xls", sheet_name = 'Sheet1')

In [4]:
def extract_amount_from_cdl_string(cdl_string):
     matches = re.search(string = cdl_string, pattern = "\#\s*(\d*\.?\d*)")
     first_match = matches.group(1)
     return float(first_match)

def extract_unit_from_cdl_string(cdl_string):
     matches = re.search(string = cdl_string, pattern = "\#\s+\d*\.?\d*\s+([a-zA-Z]+)")
     first_match = matches.group(1)
     return first_match

def get_date_from_timestamp_string(dt_string):
    return datetime.strptime(dt_string, '%b/%d/%y %H:%M:%S').date()

def enforce_types_on_keys(dat):
    # ensure type consistency
    dat["FILL_DATE"] = dat["FILL_DATE"].astype('datetime64[ns]')
    dat["NDC_DESC"] = dat["NDC_DESC"].astype('object')
    dat["UNIT"] = dat["UNIT"].astype('object')
    
    return dat

def extract_amount_unit_and_fill_date(dat):
    dat["AMOUNT"] = dat["CDL"].apply(extract_amount_from_cdl_string)
    dat["UNIT"] = dat["CDL"].apply(extract_unit_from_cdl_string)
    dat['FILL_DATE'] = dat["DISPENSE_DT_TM"].apply(get_date_from_timestamp_string)
    
    dat = enforce_types_on_keys(dat)
    
    return dat

In [5]:
def build_drug_date_spine(dat):
    """Build a dataframe and ensure that a every drug is represented for every date, even if nothing was dispensed"""

    dates = pd.date_range(dat['FILL_DATE'].min(), dat['FILL_DATE'].max(), freq="D").to_frame().rename(columns={0:"FILL_DATE"})
    dates["key"] = 1

    unique_drugs = dat[["NDC_DESC", "UNIT"]].drop_duplicates()
    unique_drugs["key"] = 1
    
    spine = pd.merge(dates, unique_drugs, on='key')[["FILL_DATE", "NDC_DESC", "UNIT"]]
    
    spine = enforce_types_on_keys(spine)
    
    return spine

In [6]:
def get_daily_totals(raw_dat):
    
    dat = extract_amount_unit_and_fill_date(raw_dat)
    
    daily_totals = dat.groupby(by=["FILL_DATE", "NDC_DESC", "UNIT"]).sum().reset_index()[["FILL_DATE","NDC_DESC","UNIT", "AMOUNT"]]

    spine = build_drug_date_spine(dat)
    
    output =  pd.merge(spine, daily_totals, how='left', on=["FILL_DATE", "NDC_DESC", "UNIT"])
    

    return output

In [7]:
daily_totals = get_daily_totals(dat)

In [8]:
daily_totals.dtypes

FILL_DATE    datetime64[ns]
NDC_DESC             object
UNIT                 object
AMOUNT              float64
dtype: object

In [9]:
example_drug = daily_totals[daily_totals["NDC_DESC"] == "atorvastatin 80 mg tablet"].set_index(["FILL_DATE", "NDC_DESC", "UNIT"]).fillna(0)
example_drug

,,,AMOUNT
FILL_DATE,NDC_DESC,UNIT,
2021-03-01,atorvastatin 80 mg tablet,EA,180.0
2021-03-02,atorvastatin 80 mg tablet,EA,0.0
2021-03-03,atorvastatin 80 mg tablet,EA,90.0
2021-03-04,atorvastatin 80 mg tablet,EA,0.0
2021-03-05,atorvastatin 80 mg tablet,EA,0.0
...,...,...,...
2021-05-27,atorvastatin 80 mg tablet,EA,90.0
2021-05-28,atorvastatin 80 mg tablet,EA,90.0
2021-05-29,atorvastatin 80 mg tablet,EA,0.0


In [10]:
example_drug.head(20)

,,,AMOUNT
FILL_DATE,NDC_DESC,UNIT,
2021-03-01,atorvastatin 80 mg tablet,EA,180.0
2021-03-02,atorvastatin 80 mg tablet,EA,0.0
2021-03-03,atorvastatin 80 mg tablet,EA,90.0
2021-03-04,atorvastatin 80 mg tablet,EA,0.0
2021-03-05,atorvastatin 80 mg tablet,EA,0.0
2021-03-06,atorvastatin 80 mg tablet,EA,90.0
2021-03-07,atorvastatin 80 mg tablet,EA,180.0
2021-03-08,atorvastatin 80 mg tablet,EA,90.0
2021-03-09,atorvastatin 80 mg tablet,EA,270.0


In [40]:
# exclude windows with no observations from the average sum calculation
def calculate_max_and_min_for_ndc_desc(dat, ndc_desc, rolling_window_days = 5, risk_factor = 1):

    sub_dat = dat[dat["NDC_DESC"] == ndc_desc]
    sub_dat = sub_dat.set_index(["FILL_DATE", "NDC_DESC", "UNIT"]).fillna(0)

    sub_dat["N_DAY_ROLLING_SUM"] = sub_dat.rolling(rolling_window_days).sum(skipna=True)

    # exclude windows with no observations from the mean calculation
    n_day_rolling_sum_average = sub_dat[sub_dat["N_DAY_ROLLING_SUM"] != 0]["N_DAY_ROLLING_SUM"].mean()

    # exclude windows with no observations from the std calculation
    n_day_rolling_sum_std = sub_dat[sub_dat["N_DAY_ROLLING_SUM"] != 0]["N_DAY_ROLLING_SUM"].std()

    potential_dispensement_days_count = len(sub_dat["AMOUNT"])
    days_dispensed = len(sub_dat["AMOUNT"][sub_dat["AMOUNT"] != 0])

    min_value = n_day_rolling_sum_average
    max_value = min_value + n_day_rolling_sum_std * risk_factor

    output = {
        "NDC Description": ndc_desc,
        "Potential Dispensed Days": potential_dispensement_days_count,
        "Risk Factor": risk_factor,
        "Rolling Window Days": rolling_window_days,
        "Mean of Rolling Sum": n_day_rolling_sum_average,
        "STD. Of Rolling Sum": n_day_rolling_sum_std,
        "Actual Dispensed Days": days_dispensed,
        "Recommended Min Level": min_value,
        "Recommended Max Level": max_value
    }

    return output

In [41]:
calculate_max_and_min_for_ndc_desc(daily_totals, "atorvastatin 80 mg tablet")

{'NDC Description': 'atorvastatin 80 mg tablet',
 'Potential Dispensed Days': 92,
 'Risk Factor': 1,
 'Rolling Window Days': 5,
 'Mean of Rolling Sum': 432.10843373493975,
 'STD. Of Rolling Sum': 199.76071124541232,
 'Actual Dispensed Days': 53,
 'Recommended Min Level': 432.10843373493975,
 'Recommended Max Level': 631.8691449803521}

In [42]:
unique_ndcs = list(daily_totals["NDC_DESC"].unique())

In [43]:
output_list = [calculate_max_and_min_for_ndc_desc(daily_totals, ndc_desc) for ndc_desc in unique_ndcs]

In [45]:
output = pd.DataFrame.from_dict(output_list)

In [46]:
output.to_csv("../data/processed/example-output-all-ndcs.csv")